In [1]:
import pandas as pd
import json
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import FastText, Word2Vec2Vec
from epochlogger import EpochLogger
from sentencegenerator import SentenceGenerator
import nltk
import re

### Load the shuffled diary entries

In [2]:
data = pd.read_csv('../data/mfp-diaries-shuffled.tsv',sep='\t',header=None,chunksize=50000)

### Construct the corpus with each sentence corresponding to an individual food name

In [3]:
# i = 0
# with open("../data/foods_as_sentences_retry.txt", "a",encoding='utf-8') as sentences:
#     for chunk in data:
#         for row in chunk.iterrows():
#             ex_diary = json.loads(row[1][2])
#             for meal in ex_diary:
#                 dish_list = []
#                 for dish in meal['dishes']:
#                     if dish['name'][0:20] == 'Quick Added Calories':
#                         provider = 'Quick Added Calories'
#                         item = 'Quick Added Calories'
#                     else:
#                         name, quant = dish['name'].split(',',maxsplit=1)
#                         try:
#                             provider, item = name.split(' - ')[0:2]
#                         except ValueError:
#                             if len(name.split(' - ')) ==1:
#                                 provider = name
#                                 item = ""
#                 sentence = provider + ' ' + item
#                 sentences.write(sentence+'\n')
#             i += 1
#             print(i,end='\r')

In [4]:
i = 0
with open("../data/foods_as_sentences_lower_w2v.txt", "a",encoding='utf-8') as sentences:
    for chunk in data:
        for row in chunk.iterrows():
            ex_diary = json.loads(row[1][2])
            for meal in ex_diary:
                for dish in meal['dishes']:
                    if dish['name'][0:20] == 'Quick Added Calories':
                        name = 'Quick Added Calories'
                    else:
                        name, quant = dish['name'].split(',',maxsplit=1)
                        tokens = re.split("[, \-!?*+()012345678~9&%=/\"#.>^<:]+",name)
                        tokens_lower =  [token.lower() for token in tokens if len(token) > 2]
                    sentence = ' '.join(tokens_lower)
                    sentences.write(sentence+'\n')
            i += 1
            print(i,end='\r')
sentences.close()
total_ex = i

In [4]:
# i = 0
# with open("../data/meals_as_sentences_lower.txt", "w",encoding='utf-8') as sentences:
#     for chunk in data:
#         for row in chunk.iterrows():
#             ex_diary = json.loads(row[1][2])
#             for meal in ex_diary:
#                 meal_sentence = ''
#                 for dish in meal['dishes']:
#                     if dish['name'][0:20] == 'Quick Added Calories':
#                         name = 'Quick Added Calories'
#                     else:
#                         name, quant = dish['name'].split(',',maxsplit=1)
#                         tokens = re.split("[, \-!?*+()012345678~9&%=/\"#.>^<:]+",name)
#                         tokens_lower =  [token.lower() for token in tokens if len(token) > 2]
#                     food_name = ' '.join(tokens_lower)
#                     meal_sentence += (food_name + ' ')
#                 sentences.write(meal_sentence+'\n')
#             i += 1
#             print(i,end='\r')
# sentences.close()
# total_ex = i

### Train Word2Vec

In [5]:
sentences = SentenceGenerator('../data/meals_as_sentences_lower.txt')
model = FastText(sentences,min_count=10,window=10)

In [13]:
sentences = SentenceGenerator('../data/foods_as_sentences_lower_w2v.txt')
model = Word2Vec(sentences,min_count=10,window=6)

In [14]:
model.save("../fasttextmodels/foods_w2v_window6_mc10.model")

In [17]:
model.wv.most_similar('raspberry')

[('rasberry', 0.8593851923942566),
 ('strawberry', 0.6730685830116272),
 ('blueberry', 0.6401907801628113),
 ('cherry', 0.6177578568458557),
 ('blackberry', 0.613835334777832),
 ('peach', 0.555031418800354),
 ('berry', 0.5057841539382935),
 ('pomegranate', 0.4807957112789154),
 ('stawberry', 0.47010713815689087),
 ('apricot', 0.45199546217918396)]